In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    
    # Iterate over each GPU
    for i in range(num_gpus):
        # Get the GPU name and ID
        gpu_name = torch.cuda.get_device_name(i)
        gpu_id = torch.cuda.get_device_properties(i)
        
        # Print the GPU name and ID
        print(f"GPU {i}: {gpu_name} (ID: {gpu_id})")
else:
    print("No GPUs available.")


GPU 0: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 1: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 2: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 3: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 4: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 5: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))
GPU 6: NVIDIA RTX A5000 (ID: _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24247MB, multi_processor_count=64))

In [18]:
# set cuda:1 as the device
device = torch.device('cuda:1')

class FreezeLinear(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.W = nn.Parameter(torch.ones(5000, 5000), requires_grad=True)
        self.linear = nn.Linear(5000, 5000)
        
    def forward(self, x):
        y = F.linear(x, self.W)
        y = self.linear(y)
        return y

In [19]:
for i in range(1000):
    model = FreezeLinear()
    model = model.to(device)
    input_x = torch.randn(1000, 5000).to(device)
    y = model(input_x)

In [8]:
import torch

def fasterKDP(x, a, b):
    x_shape = x.shape
    a_shape = a.shape
    b_shape = b.shape
    
    
    assert a_shape[0] * b_shape[0] == x_shape[-1], "The shapes of the input tensor and the factors are not compatible"
    # change x[-1] into a[0] b[0]
    x = x.view(-1, a_shape[0], b_shape[0])    
    x = x @ b
    x = torch.permute(x, (0, 2, 1)).contiguous()
    x = x @ a
    x = torch.permute(x, (0, 2, 1)).contiguous()
    
    y_shape = [*x_shape[:-1], a_shape[-1] * b_shape[-1]]
    x = x.view(y_shape)
    return x

x = torch.randn(2, 3, 12)
a = torch.randn(3, 4)
b = torch.randn(4, 5)
y = fasterKDP(x, a, b)
y.shape


torch.Size([2, 3, 20])

In [3]:
import torch
a = torch.randn(1,3, 2, 3)
w = torch.randn(3, 4)
c = a @ w
c.shape 

torch.Size([1, 3, 2, 4])

In [1]:
from local_models.KronLinear import *

ImportError: cannot import name 'KronLinear' from partially initialized module 'local_models.KronLinear' (most likely due to a circular import) (/home/zhu.3723/kronvit/local_models/KronLinear.py)